# Cloud Tools - Firebase

In [1]:
import os
import sys
from pymagic.cloud_tools import Firebase
import firebase_admin
from firebase_admin import firestore

# App Initialization

To interact with Firebase's SDK through Python, we have to create an 'app' instance of our Firebase project.

For this you specify the URLs of your project as well as your authentication token.

In [2]:
firebase_app_client = \
    Firebase.initapp(
        storage_url=os.environ['example_firebase_storage_url'],
        db_url=os.environ['example_firebase_db_url'],
        token_path=os.environ['example_firebase_token_path']
)

firestore_client = \
    firebase_admin.firestore.client(
        app=firebase_app_client
)

database = firestore.client()

# FireStorage

FireStorage is Firebase's file storage service, similar to AWS's S3.

## Send File to Bucket

In [3]:
if sys.platform == "linux":
    sav_dir = "/home/collier/test_folder/"
else:
    sav_dir = "/Users/collier/Downloads/"

Firebase.file_to_bucket(
    blob_name="test.png",
    file_path=sav_dir + "test.png",
    content_type='image/png',
    metadata_d={"test_key": 'test_val'}
)

## List Bucket Objects

This function returns a list of bucket objects. From these objects you can retrieve object names, time_created, metadata and more!  Let's print the names of our bucket objects.

In [4]:
bucket_blobs = Firebase.list_bucket_objects()

In [5]:
[x.name for x in bucket_blobs[-1:]]

['test.png']

## Delete Bucket Object

In [12]:
Firebase.delete_bucket_object(
    blob_name="test.png"
)

Blob test.png deleted.


# FireStore

FireStore is Firebase's flagship collection/document-based NoSQL database. Similar to AWS's DynamoDB.

## FireStore Objects

From [this Firbease blog](https://firebase.googleblog.com/2019/06/understanding-collection-group-queries.html): "In Cloud Firestore, your data is divided up into documents and collections. Documents often point to subcollections that contain other documents, like in this example, where each restaurant document contains a subcollection with all the reviews of that restaurant.

So in order to interact with Firestore, we need to specify the sequence of objects needed to reach the final object we are creating/updating/deleting.

## Creating/Updating Documents

Let's create some objects in our database.

###  Inside a Collection

When we add a document to our database, we need to specify that the parent object type is a 'collection' as well as that collection's name.

If the collection does not exist, Firebase will create it for us.

In [32]:
#define the collection/document path
obj_types = ["collection","document"]
obj_names = ["test_collection","test_document"]

#### Creating

In [33]:
Firebase.load_delete_firestore(
    method="insert",
    obj_types=obj_types,
    obj_names=obj_names,
    #new document contents
    d={"field":"val"}
)

collection
document


update_time {
  seconds: 1603053909
  nanos: 777949000
}

#### Updating

If we want to update a document, we have several options.  

First, let's perform a straight update which completelt overwrites the document's data.

Here we need to specify the tree of objects that lead to the document to update:

In [34]:
#define the collection/document path
obj_types = ["collection","document"]
obj_names = ["test_collection","test_document"]

In [35]:
Firebase.load_delete_firestore(
    method="update",
    obj_types=obj_types,
    obj_names=obj_names,
    #updated document contents
    d={"field_updated":"val_updated"}
)

collection
document
no key constraint, running straight update...


update_time {
  seconds: 1603053947
  nanos: 517773000
}

#### Updating with a Constraint

What if we don't want to perform am update, but only want to update a document if it meets some constraint, such as a field having a certain value.  This will search for documents that have the key constraint.

To do that, we just need to include a 'constraint_key' and its value in our function call.

Now we will only update documents that meet this condition.

Notice that we left off the 'doc_key' key and value in our dictionary since this operation will keep the existing 'doc_key' in the database and simply update the 'doc_data' values.

Also, notice that in our collection/document path, we fall back to our root collection containing the documents we are seeing to update.

In [36]:
#define the collection/document path
obj_types = ["collection"]
obj_names = ["test_collection"]

In [37]:
Firebase.load_delete_firestore(
    method="update",
    obj_types=obj_types,
    obj_names=obj_names,
    constraint_key="field_updated",
    constraint_val="val_updated",
    #updated document contents
    d={"field_updated":"val_updated_new"}
)

collection
update with key constraint...
updating 1 documents that met key constraint...
updating document 0


[update_time {
   seconds: 1603053994
   nanos: 871689000
 }]

#### Upserting with a Constraint

What if we attempt to update a document using a key constraint, but no document exists?  In that case we would want to insert it into the collection.  This type of operation is called an **Upsert**.

To accomplish this, simply add a value to the 'upsert_doc_name' in the function call, this will create a new document with that name if the constraint key is not found.

In [38]:
#define the collection/document path
obj_types = ["collection"]
obj_names = ["test_collection"]

In [39]:
Firebase.load_delete_firestore(
    method="update",
    obj_types=obj_types,
    obj_names=obj_names,
    constraint_key="field_updated",
    constraint_val="val_update_old", #doesnt exist
    upsert_doc_name="test_doc_new", #new doc name
    #updated document contents
    d={"field":"val_upsert"}
)

TypeError: load_delete_firestore() got an unexpected keyword argument 'upsert_doc_name'

### Inside a Subcollection

A sub-collection is a collection that lives inside a document.  We can create a sub collection by specifying its name and the names of the parent collection and document.  

Notice we specify the object names based on the order of their heirarchy in the database.  This is not required but it helps to undersrand where data is being inserted/updated.

In [11]:
Firebase.load_delete_firestore(
    method="insert",
    parent_object_type="sub_collection",
    parent_collection_name="test_collection",
    parent_document_name="test_doc",
    parent_sub_collection_name="test_sub_collection",
    d={
        "doc_key":"test_sub_collection_doc",
        "doc_data":{"field":"val"}
      }
)

target object: sub_collection...
operation method: insert...


update_time {
  seconds: 1601163959
  nanos: 769187000
}

Let's do the same operations we did on collection documents, this time on sub-collection documents.

In [12]:
#straight update
Firebase.load_delete_firestore(
    method="update",
    parent_object_type="sub_collection",
    parent_collection_name="test_collection",
    parent_document_name="test_doc",
    parent_sub_collection_name="test_sub_collection",    
    d={
        "doc_key":"test_sub_collection_doc",
        "doc_data":{"field":"val_updated"}
      }
)

target object: sub_collection...
operation method: update...
no key constraint, running straight update...


update_time {
  seconds: 1601163959
  nanos: 875315000
}

In [13]:
#update with constraint
Firebase.load_delete_firestore(
    method="update",
    parent_object_type="sub_collection",
    parent_collection_name="test_collection",
    parent_document_name="test_doc",
    parent_sub_collection_name="test_sub_collection", 
    constraint_key="field",
    constraint_val="val_updated",
    #updated document contents
    d={
        "doc_data":{"field_updated":"val_updated_new"}
      }
)

target object: sub_collection...
operation method: update...
update with key constraint...
updating 1 documents that met key constraint...
updating document 0


[update_time {
   seconds: 1601163960
   nanos: 29143000
 }]

In [14]:
#upsert with constraint
Firebase.load_delete_firestore(
    method="update",
    parent_object_type="sub_collection",
    parent_collection_name="test_collection",
    parent_document_name="test_doc",
    parent_sub_collection_name="test_sub_collection", 
    constraint_key="field_updated",
    constraint_val="val_updated_old",
    upsert=True,
    #updated document contents
    d={
        "doc_key":"upsert_test",
        "doc_data":{"field":"val_upsert"}
      }
)

target object: sub_collection...
operation method: update...
update with key constraint...
no key constraint results found...
upsert triggered...inserting document...


update_time {
  seconds: 1601163960
  nanos: 191911000
}

## Deleting Objects

When deleting objects in Firebase, we can do much of the same types of operations we did when we were updating objects.

Let's create a few more objects to delete.

In [15]:
#single field to delete in a document
Firebase.load_delete_firestore(
    method="insert",
    parent_object_type="collection",
    parent_collection_name="test_collection",
    #new document contents
    d={
        "doc_key":"delete_field",
        "doc_data":{"field_to_delete":"val"}
      }
)

#single document to delete by name
Firebase.load_delete_firestore(
    method="insert",
    parent_object_type="collection",
    parent_collection_name="test_collection",
    #new document contents
    d={
        "doc_key":"delete_by_name",
        "doc_data":{"field":"val"}
      }
)

#single document to delete by key constraint
Firebase.load_delete_firestore(
    method="insert",
    parent_object_type="collection",
    parent_collection_name="test_collection",
    #new document contents
    d={
        "doc_key":"delete_by_key_constraint",
        "doc_data":{"field":"delete_me"}
      }
)

#single document to delete key constraint 
# within a subcollection
Firebase.load_delete_firestore(
    method="insert",
    parent_object_type="sub_collection",
    parent_collection_name="test_collection",
    parent_document_name="test_doc",
    parent_sub_collection_name="test_sub_collection",
    d={
        "doc_key":"delete_by_key_constraint",
        "doc_data":{"field":"delete_me"}
      }
)

target object: collection...
operation method: insert...
target object: collection...
operation method: insert...
target object: collection...
operation method: insert...
target object: sub_collection...
operation method: insert...


update_time {
  seconds: 1601163960
  nanos: 537962000
}

### Deleting a Field

Let's delete a single field in a document

In [16]:
Firebase.load_delete_firestore(
    method="delete",
    parent_object_type="field",
    parent_collection_name="test_collection",
    parent_document_name="delete_field",
    constraint_key="field_to_delete"
)

target object: field...
operation method: delete...
deleting document field: field_to_delete...


update_time {
  seconds: 1601163960
  nanos: 634411000
}

### Deleting Document by Name

Let's delete a document by name.

In [17]:
Firebase.load_delete_firestore(
    method="delete",
    parent_object_type="document",
    parent_collection_name="test_collection",
    parent_document_name="delete_by_name",
)

target object: document...
operation method: delete...
deleting document...


seconds: 1601163960
nanos: 732442000

### Deleting  Document by Key Constraint

In [18]:
#collection
Firebase.load_delete_firestore(
    method="delete",
    parent_object_type="collection",
    parent_collection_name="test_collection",
    constraint_key="field",
    constraint_val="delete_me"
)

target object: collection...
operation method: delete...
deleting documents with key constraint...
Deleting doc: delete_by_key_constraint


In [19]:
#subcollection
Firebase.load_delete_firestore(
    method="delete",
    parent_object_type="sub_collection",
    parent_collection_name="test_collection",
    parent_document_name="test_doc",
    parent_sub_collection_name="test_sub_collection",
    constraint_key="field",
    constraint_val="delete_me"
)

target object: sub_collection...
operation method: delete...
deleting documents with key constraint...
Deleting doc: delete_by_key_constraint


### Deleting All Documents

To delete all documents in a collection or subcollection, we just need to leave off the key constraint argument.

You can also specify a 'document_delete_batch_size' to avoid memory errors if the size of the collection is to big.  The function will call itself again if the deleted document count exceeds this limit.

In [20]:
#subcollection
Firebase.load_delete_firestore(
    method="delete",
    parent_object_type="sub_collection",
    parent_collection_name="test_collection",
    parent_document_name="test_doc",
    parent_sub_collection_name="test_sub_collection",
    document_delete_batch_size=1
#     constraint_key="field",
#     constraint_val="delete_me"
)

target object: sub_collection...
operation method: delete...
Deleting doc: test_sub_collection_doc
target object: sub_collection...
operation method: delete...
Deleting doc: upsert_test
target object: sub_collection...
operation method: delete...
No documents found...exiting...


In [21]:
#collection
Firebase.load_delete_firestore(
    method="delete",
    parent_object_type="collection",
    parent_collection_name="test_collection",
    document_delete_batch_size=1
#     constraint_key="field",
#     constraint_val="delete_me"
)

target object: collection...
operation method: delete...
Deleting doc: delete_field
target object: collection...
operation method: delete...
Deleting doc: test_doc
target object: collection...
operation method: delete...
Deleting doc: upsert_test
target object: collection...
operation method: delete...
No documents found...exiting...
